In [48]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import seaborn as sns

In [29]:
# Load the data
ground_truths_path = 'ground_truths/ground_truths.csv'  # Replace with your local path
predicted_times_path = 'results/files/predicted_times.csv'  # Replace with your local path

ground_truths = pd.read_csv(ground_truths_path)
predicted_times = pd.read_csv(predicted_times_path)

# Merge ground truths with predictions based on the image name
comparison = pd.merge(predicted_times, ground_truths, 
                      left_on='Image Name', right_on='Watch', how='inner')


In [50]:
comparison

In [57]:
# Define a function to classify as successful or failed based on the "Predicted Time"
def classify_prediction(predicted_time):
    if isinstance(predicted_time, str) and ("failed" in predicted_time.lower() or "detection failed" in predicted_time.lower()):
        return "Failed"
    else:
        return "Successful"

# Apply the classification function to the 'Predicted Time' column
comparison['Prediction Status'] = comparison['Predicted Time'].apply(classify_prediction)

# Calculate the number of successful and failed predictions
successful_predictions = (comparison['Prediction Status'] == "Successful").sum()
failed_predictions = (comparison['Prediction Status'] == "Failed").sum()

# Calculate the ratio of successful to failed detections
total_predictions = len(comparison)
success_ratio = successful_predictions / total_predictions * 100
failure_ratio = failed_predictions / total_predictions * 100

# Output the results
print(f"Total Predictions: {total_predictions}")
print(f"Successful Predictions: {successful_predictions} ({success_ratio:.2f}%)")
print(f"Failed Predictions: {failed_predictions} ({failure_ratio:.2f}%)")


In [58]:
import matplotlib.pyplot as plt

# Data for the pie chart
labels = ['Successful Predictions', 'Failed Predictions']
sizes = [successful_predictions, failed_predictions]
colors = ['#4CAF50', '#F44336']  # Green for successful, red for failed
explode = (0.1, 0)  # Slightly "explode" the successful slice to highlight it

# Plotting the pizza chart
plt.figure(figsize=(7, 7))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140, 
        shadow=True, wedgeprops={'edgecolor': 'black'})
plt.title('Prediction Success vs Failure')
plt.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
plt.show()


In [51]:
# Convert times to seconds for plotting
def time_to_seconds(time_str):
    """Convert time string (HH:MM:SS) to total seconds."""
    try:
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    except:
        return None

# Add conversions for seconds, minutes, and hours
comparison['Ground Truth Seconds'] = comparison['Time'].apply(time_to_seconds)
comparison['Predicted Seconds'] = comparison['Predicted Time'].apply(time_to_seconds)

# Convert seconds to minutes and hours
comparison['Ground Truth Minutes'] = comparison['Ground Truth Seconds'] / 60
comparison['Predicted Minutes'] = comparison['Predicted Seconds'] / 60

comparison['Ground Truth Hours'] = comparison['Ground Truth Seconds'] / 3600
comparison['Predicted Hours'] = comparison['Predicted Seconds'] / 3600

# Filter out invalid rows
valid_comparison = comparison.dropna(subset=['Ground Truth Seconds', 'Predicted Seconds'])

# Calculate time differences (absolute value) for seconds, minutes, and hours
valid_comparison['Time Difference (Seconds)'] = (valid_comparison['Predicted Seconds'] - valid_comparison['Ground Truth Seconds']).abs()
valid_comparison['Time Difference (Minutes)'] = (valid_comparison['Predicted Minutes'] - valid_comparison['Ground Truth Minutes']).abs()
valid_comparison['Time Difference (Hours)'] = (valid_comparison['Predicted Hours'] - valid_comparison['Ground Truth Hours']).abs()



In [52]:
valid_comparison

In [38]:
# Plot the time differences for seconds, minutes, and hours
plt.figure(figsize=(18, 6))

# Scatter plot for seconds
plt.subplot(1, 3, 1)
plt.scatter(valid_comparison.index, valid_comparison['Time Difference (Seconds)'], color='blue', alpha=0.5)
plt.xlabel('Image Index')
plt.ylabel('Time Difference (Seconds)')
plt.title('Time Differences (Seconds)')

# Scatter plot for minutes
plt.subplot(1, 3, 2)
plt.scatter(valid_comparison.index, valid_comparison['Time Difference (Minutes)'], color='green', alpha=0.5)
plt.xlabel('Image Index')
plt.ylabel('Time Difference (Minutes)')
plt.title('Time Differences (Minutes)')

# Scatter plot for hours
plt.subplot(1, 3, 3)
plt.scatter(valid_comparison.index, valid_comparison['Time Difference (Hours)'], color='red', alpha=0.5)
plt.xlabel('Image Index')
plt.ylabel('Time Difference (Hours)')
plt.title('Time Differences (Hours)')

# Adjust layout for better spacing
plt.tight_layout()
plt.show()


In [47]:
# Define thresholds for each unit
threshold_seconds = 60  
threshold_minutes = 1   
threshold_hours = 0.0167  # approximately 1 minute in hours

# Calculate correct predictions for each unit
correct_seconds = valid_comparison['Time Difference (Seconds)'] <= threshold_seconds
correct_minutes = valid_comparison['Time Difference (Minutes)'] <= threshold_minutes
correct_hours = valid_comparison['Time Difference (Hours)'] <= threshold_hours

# Calculate accuracy percentages
accuracy_seconds = (correct_seconds.sum() / len(valid_comparison)) * 100
accuracy_minutes = (correct_minutes.sum() / len(valid_comparison)) * 100
accuracy_hours = (correct_hours.sum() / len(valid_comparison)) * 100

# Print the accuracy percentages
print(f"Accuracy (Seconds): {accuracy_seconds:.2f}%")
print(f"Accuracy (Minutes): {accuracy_minutes:.2f}%")
print(f"Accuracy (Hours): {accuracy_hours:.2f}%")

# Bar plot for correct vs incorrect predictions for each unit
plt.figure(figsize=(12, 6))

# Seconds
plt.subplot(1, 3, 1)
plt.bar(['Correct', 'Incorrect'], [correct_seconds.sum(), (~correct_seconds).sum()], color=['green', 'red'])
plt.title(f'Accuracy (Seconds): {accuracy_seconds:.2f}%')
plt.ylabel('Number of Predictions')
plt.xlabel('Seconds Threshold')

# Minutes
plt.subplot(1, 3, 2)
plt.bar(['Correct', 'Incorrect'], [correct_minutes.sum(), (~correct_minutes).sum()], color=['green', 'red'])
plt.title(f'Accuracy (Minutes): {accuracy_minutes:.2f}%')
plt.xlabel('Minutes Threshold')

# Hours
plt.subplot(1, 3, 3)
plt.bar(['Correct', 'Incorrect'], [correct_hours.sum(), (~correct_hours).sum()], color=['green', 'red'])
plt.title(f'Accuracy (Hours): {accuracy_hours:.2f}%')
plt.xlabel('Hours Threshold')

plt.tight_layout()
plt.show()


# Standard Deviation 

In [40]:
# Calculate the standard deviation of the time differences
std_dev_seconds = valid_comparison['Time Difference (Seconds)'].std()
std_dev_minutes = valid_comparison['Time Difference (Minutes)'].std()
std_dev_hours = valid_comparison['Time Difference (Hours)'].std()

# Print the standard deviations
print(f"Standard Deviation of Time Differences:")
print(f"  Seconds: {std_dev_seconds:.2f} seconds")
print(f"  Minutes: {std_dev_minutes:.2f} minutes")
print(f"  Hours: {std_dev_hours:.4f} hours")  # Higher precision for small hour values


In [41]:
import matplotlib.pyplot as plt

# Plot histograms for seconds, minutes, and hours
plt.figure(figsize=(18, 6))

# Histogram for seconds
plt.subplot(1, 3, 1)
plt.hist(valid_comparison['Time Difference (Seconds)'], bins=50, color='blue', edgecolor='black')
plt.title('Time Differences (Seconds)')
plt.xlabel('Time Difference (Seconds)')
plt.ylabel('Frequency')

# Histogram for minutes
plt.subplot(1, 3, 2)
plt.hist(valid_comparison['Time Difference (Minutes)'], bins=50, color='green', edgecolor='black')
plt.title('Time Differences (Minutes)')
plt.xlabel('Time Difference (Minutes)')
plt.ylabel('Frequency')

# Histogram for hours
plt.subplot(1, 3, 3)
plt.hist(valid_comparison['Time Difference (Hours)'], bins=50, color='red', edgecolor='black')
plt.title('Time Differences (Hours)')
plt.xlabel('Time Difference (Hours)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Plot boxplots for seconds, minutes, and hours
plt.figure(figsize=(12, 6))

# Boxplot for seconds
plt.subplot(1, 3, 1)
plt.boxplot(valid_comparison['Time Difference (Seconds)'], vert=False, patch_artist=True, boxprops=dict(facecolor='blue', color='black'))
plt.title('Boxplot of Time Differences (Seconds)')
plt.xlabel('Time Difference (Seconds)')

# Boxplot for minutes
plt.subplot(1, 3, 2)
plt.boxplot(valid_comparison['Time Difference (Minutes)'], vert=False, patch_artist=True, boxprops=dict(facecolor='green', color='black'))
plt.title('Boxplot of Time Differences (Minutes)')
plt.xlabel('Time Difference (Minutes)')

# Boxplot for hours
plt.subplot(1, 3, 3)
plt.boxplot(valid_comparison['Time Difference (Hours)'], vert=False, patch_artist=True, boxprops=dict(facecolor='red', color='black'))
plt.title('Boxplot of Time Differences (Hours)')
plt.xlabel('Time Difference (Hours)')

plt.tight_layout()
plt.show()


In [42]:
# Define outlier thresholds
outlier_threshold_seconds = 3600  # in seconds
outlier_threshold_minutes = outlier_threshold_seconds / 60  # in minutes
outlier_threshold_hours = outlier_threshold_seconds / 3600  # in hours

# Filter out time differences above the threshold for each unit
filtered_comparison_seconds = valid_comparison[valid_comparison['Time Difference (Seconds)'].abs() <= outlier_threshold_seconds]
filtered_comparison_minutes = valid_comparison[valid_comparison['Time Difference (Minutes)'].abs() <= outlier_threshold_minutes]
filtered_comparison_hours = valid_comparison[valid_comparison['Time Difference (Hours)'].abs() <= outlier_threshold_hours]

# Recalculate the standard deviation without outliers
filtered_std_dev_seconds = filtered_comparison_seconds['Time Difference (Seconds)'].std()
filtered_std_dev_minutes = filtered_comparison_minutes['Time Difference (Minutes)'].std()
filtered_std_dev_hours = filtered_comparison_hours['Time Difference (Hours)'].std()

# Print the recalculated standard deviations
print("Standard Deviation without Outliers:")
print(f"  Seconds: {filtered_std_dev_seconds:.2f} seconds")
print(f"  Minutes: {filtered_std_dev_minutes:.2f} minutes")
print(f"  Hours: {filtered_std_dev_hours:.4f} hours")



# Mean Absolute Error

In [43]:
import numpy as np

def mean_absolute_error_with_threshold(y_true, y_pred, threshold=None):
    """
    Calculate the Mean Absolute Error (MAE) with an optional threshold for outliers.
    
    Args:
    - y_true: Array-like of ground truth values.
    - y_pred: Array-like of predicted values.
    - threshold: Optional; exclude errors above this threshold.
    
    Returns:
    - Mean absolute error, or None if no data remains after thresholding.
    """
    # Calculate the absolute errors
    errors = np.abs(y_true - y_pred)
    
    # Apply threshold to filter out outliers if provided
    if threshold is not None:
        errors = errors[errors <= threshold]
    
    # Return the mean of the errors, or None if no valid errors remain
    return np.mean(errors) if len(errors) > 0 else None

# Threshold values for each unit
threshold_seconds = 3600  # in seconds
threshold_minutes = threshold_seconds / 60  # in minutes
threshold_hours = threshold_seconds / 3600  # in hours

# Calculate MAE for each unit
mae_seconds = mean_absolute_error_with_threshold(
    valid_comparison['Ground Truth Seconds'], 
    valid_comparison['Predicted Seconds'], 
    threshold=threshold_seconds
)

mae_minutes = mean_absolute_error_with_threshold(
    valid_comparison['Ground Truth Minutes'], 
    valid_comparison['Predicted Minutes'], 
    threshold=threshold_minutes
)

mae_hours = mean_absolute_error_with_threshold(
    valid_comparison['Ground Truth Hours'], 
    valid_comparison['Predicted Hours'], 
    threshold=threshold_hours
)

# Print results
print(f"Mean Absolute Error (MAE) with threshold:")
print(f"  Seconds: {mae_seconds:.2f} seconds")
print(f"  Minutes: {mae_minutes:.2f} minutes")
print(f"  Hours: {mae_hours:.4f} hours")



# Bias

In [44]:
# Calculate the bias (mean error) for each unit
bias_seconds = (valid_comparison['Predicted Seconds'] - valid_comparison['Ground Truth Seconds']).mean()
bias_minutes = (valid_comparison['Predicted Minutes'] - valid_comparison['Ground Truth Minutes']).mean()
bias_hours = (valid_comparison['Predicted Hours'] - valid_comparison['Ground Truth Hours']).mean()

# Print the biases
print("Bias (Mean Error):")
print(f"  Seconds: {bias_seconds:.2f} seconds")
print(f"  Minutes: {bias_minutes:.2f} minutes")
print(f"  Hours: {bias_hours:.4f} hours")


# Error Distribution

In [45]:
# Calculate prediction errors for each time unit
valid_comparison['Error (Seconds)'] = valid_comparison['Predicted Seconds'] - valid_comparison['Ground Truth Seconds']
valid_comparison['Error (Minutes)'] = valid_comparison['Predicted Minutes'] - valid_comparison['Ground Truth Minutes']
valid_comparison['Error (Hours)'] = valid_comparison['Predicted Hours'] - valid_comparison['Ground Truth Hours']

# Individual density plots for better visualization
fig, axes = plt.subplots(3, 1, figsize=(12, 16), sharex=False)

# Seconds
sns.kdeplot(valid_comparison['Error (Seconds)'], color='blue', fill=True, alpha=0.3, ax=axes[0])
axes[0].set_title('Density Plot of Prediction Errors (Seconds)')
axes[0].axvline(0, color='black', linestyle='--', label='Zero Error Line')
axes[0].legend()

# Minutes
sns.kdeplot(valid_comparison['Error (Minutes)'], color='green', fill=True, alpha=0.3, ax=axes[1])
axes[1].set_title('Density Plot of Prediction Errors (Minutes)')
axes[1].axvline(0, color='black', linestyle='--', label='Zero Error Line')
axes[1].legend()

# Hours
sns.kdeplot(valid_comparison['Error (Hours)'], color='red', fill=True, alpha=0.3, ax=axes[2])
axes[2].set_title('Density Plot of Prediction Errors (Hours)')
axes[2].axvline(0, color='black', linestyle='--', label='Zero Error Line')
axes[2].legend()

# Add labels
for ax in axes:
    ax.set_xlabel('Error')
    ax.set_ylabel('Density')

plt.tight_layout()
plt.show()




# Correlation Coefficient

In [46]:
# Calculate Pearson correlation coefficients for each unit
correlation_seconds = valid_comparison['Ground Truth Seconds'].corr(valid_comparison['Predicted Seconds'])
correlation_minutes = valid_comparison['Ground Truth Minutes'].corr(valid_comparison['Predicted Minutes'])
correlation_hours = valid_comparison['Ground Truth Hours'].corr(valid_comparison['Predicted Hours'])

# Print the correlation coefficients
print(f"Correlation Coefficient:")
print(f"  Seconds: {correlation_seconds:.4f}")
print(f"  Minutes: {correlation_minutes:.4f}")
print(f"  Hours: {correlation_hours:.4f}")


# Bounding Boxes

In [ ]:
def create_dataset(original_dir, predicted_dir, output_csv):
    """Creates a dataset comparing original and predicted bounding boxes."""
    dataset = []

    # Iterate over all original files in the directory
    for original_file in os.listdir(original_dir):
        if original_file.endswith('.txt'):
            # Remove the '.txt' extension for the original file name
            original_file_name = original_file.replace('.txt', '')
            
            # Get the corresponding predicted json file
            base_filename = original_file_name + '_detection.json'
            predicted_file = os.path.join(predicted_dir, base_filename)
            
            # Read the original and predicted bounding boxes
            original_bboxes = read_original_bboxes(os.path.join(original_dir, original_file))
            predicted_bboxes = read_predicted_bboxes(predicted_file)
            
            # Exclude 'center' (index 4) and 'seconds' (index 3) bounding boxes
            filtered_original = [original_bboxes[i] for i in [0, 1, 2, 5]]  # keep indices 0, 1, 2, 5
            filtered_predicted = [predicted_bboxes[i] for i in [0, 1, 2, 5]]  # keep indices 0, 1, 2, 5
            
            # Combine the file name, original and predicted bounding boxes into a row
            row = [original_file_name] + [coord for bbox in filtered_original for coord in bbox] + \
                  [coord for bbox in filtered_predicted for coord in bbox]
            dataset.append(row)

    # Define the column names for the filtered dataset
    columns = ['original_file_name',
               'circle_xmin', 'circle_ymin', 'circle_xmax', 'circle_ymax',
               'hours_xmin', 'hours_ymin', 'hours_xmax', 'hours_ymax',
               'minutes_xmin', 'minutes_ymin', 'minutes_xmax', 'minutes_ymax',
               '12_xmin', '12_ymin', '12_xmax', '12_ymax',
               'circle_predicted_xmin', 'circle_predicted_ymin', 'circle_predicted_xmax', 'circle_predicted_ymax',
               'hours_predicted_xmin', 'hours_predicted_ymin', 'hours_predicted_xmax', 'hours_predicted_ymax',
               'minutes_predicted_xmin', 'minutes_predicted_ymin', 'minutes_predicted_xmax', 'minutes_predicted_ymax',
               '12_predicted_xmin', '12_predicted_ymin', '12_predicted_xmax', '12_predicted_ymax']

    # Convert the dataset to a pandas DataFrame
    df = pd.DataFrame(dataset, columns=columns)
    
    # Save the dataset to a CSV file
    df.to_csv(output_csv, index=False)
    print(f"Dataset saved to {output_csv}")

    return df

# Define paths
original_dir = 'dataset/labels/val/'
predicted_dir = 'detections/'
output_csv = 'bounding_boxes_comparison.csv'

# Create the dataset
df = create_dataset(original_dir, predicted_dir, output_csv)




Dataset saved to bounding_boxes_comparison.csv


In [31]:
# Update column names for actual and predicted bounding boxes based on the new names
actual_categories = ['circle', 'hours', 'minutes', '12']
deviation_results_updated = {}

# Calculate deviations for each category
for category in actual_categories:
    # Get actual and predicted columns for the current category
    actual_cols = [f'{category}_{dim}' for dim in ['xmin', 'ymin', 'xmax', 'ymax']]
    predicted_cols = [f'{category}_predicted_{dim}' for dim in ['xmin', 'ymin', 'xmax', 'ymax']]
    
    # Calculate absolute deviations
    deviations = abs(df[actual_cols].values - df[predicted_cols].values)
    
    # Store the mean deviation for this category
    deviation_results_updated[category] = deviations.mean(axis=0)
    


# Convert results into a DataFrame for better readability
deviation_summary_updated = pd.DataFrame.from_dict(
    deviation_results_updated, orient='index', columns=['xmin', 'ymin', 'xmax', 'ymax']
)

deviation_summary_updated


,xmin,ymin,xmax,ymax
circle,5.073543,4.996064,10.634500,10.476149
hours,7.303338,7.227620,7.754451,7.700249
minutes,7.252006,7.035207,8.184950,7.960918
12,7.829446,7.051932,8.138195,7.226047


In [32]:
df

,original_file_name,circle_xmin,circle_ymin,circle_xmax,circle_ymax,hours_xmin,hours_ymin,hours_xmax,hours_ymax,minutes_xmin,...,hours_predicted_xmax,hours_predicted_ymax,minutes_predicted_xmin,minutes_predicted_ymin,minutes_predicted_xmax,minutes_predicted_ymax,12_predicted_xmin,12_predicted_ymin,12_predicted_xmax,12_predicted_ymax
0,watch100_rotated_-270,0.473629,0.500000,0.508439,0.514768,0.449367,0.537975,0.075949,0.109705,0.434599,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,watch100_rotated_-90,0.526371,0.500000,0.508439,0.514768,0.550633,0.462025,0.075949,0.109705,0.565401,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,watch100_rotated_180,0.500000,0.526371,0.514768,0.508439,0.537975,0.550633,0.109705,0.075949,0.434599,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,watch10_rotated_-270,0.460625,0.511250,0.326250,0.332500,0.431250,0.536250,0.057500,0.052500,0.440625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,watch10_rotated_-90,0.539375,0.488750,0.326250,0.332500,0.568750,0.463750,0.057500,0.052500,0.559375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,watch99_rotated_-90,0.551688,0.474684,0.487342,0.481013,0.572785,0.439873,0.069620,0.086498,0.581224,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396,watch99_rotated_180,0.525316,0.551688,0.481013,0.487342,0.560127,0.572785,0.086498,0.069620,0.464135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397,watch9_rotated_-270,0.504861,0.503472,0.329167,0.318056,0.482639,0.538194,0.048611,0.062500,0.465278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
398,watch9_rotated_-90,0.495139,0.496528,0.329167,0.318056,0.517361,0.461806,0.048611,0.062500,0.534722,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Função para calcular o desvio médio absoluto entre os valores originais e previstos
def calculate_row_deviation(row):
    deviation = 0
    classes = ['circle', 'hours', 'minutes', '12']  # Classes disponíveis
    
    for class_name in classes:
        # Índices para valores originais e previstos da classe atual
        original_cols = [f"{class_name}_xmin", f"{class_name}_ymin", f"{class_name}_xmax", f"{class_name}_ymax"]
        predicted_cols = [f"{class_name}_predicted_xmin", f"{class_name}_predicted_ymin", f"{class_name}_predicted_xmax", f"{class_name}_predicted_ymax"]
        
        # Calcular a soma dos desvios absolutos para esta classe
        deviation += sum(abs(row[orig] - row[pred]) 
                         for orig, pred in zip(original_cols, predicted_cols)
                         if not pd.isna(row[orig]) and not pd.isna(row[pred]))
    
    # Retornar o desvio médio (normalizado pelo número de valores por classe * 6 classes)
    return deviation / (4 * len(classes))


# Adicionar uma nova coluna com o desvio para cada linha
df['Deviation'] = df.apply(calculate_row_deviation, axis=1)

df

,original_file_name,circle_xmin,circle_ymin,circle_xmax,circle_ymax,hours_xmin,hours_ymin,hours_xmax,hours_ymax,minutes_xmin,...,hours_predicted_ymax,minutes_predicted_xmin,minutes_predicted_ymin,minutes_predicted_xmax,minutes_predicted_ymax,12_predicted_xmin,12_predicted_ymin,12_predicted_xmax,12_predicted_ymax,Deviation
0,watch100_rotated_-270,0.473629,0.500000,0.508439,0.514768,0.449367,0.537975,0.075949,0.109705,0.434599,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.328455
1,watch100_rotated_-90,0.526371,0.500000,0.508439,0.514768,0.550633,0.462025,0.075949,0.109705,0.565401,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.375132
2,watch100_rotated_180,0.500000,0.526371,0.514768,0.508439,0.537975,0.550633,0.109705,0.075949,0.434599,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.371967
3,watch10_rotated_-270,0.460625,0.511250,0.326250,0.332500,0.431250,0.536250,0.057500,0.052500,0.440625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289922
4,watch10_rotated_-90,0.539375,0.488750,0.326250,0.332500,0.568750,0.463750,0.057500,0.052500,0.559375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.339453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,watch99_rotated_-90,0.551688,0.474684,0.487342,0.481013,0.572785,0.439873,0.069620,0.086498,0.581224,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.364847
396,watch99_rotated_180,0.525316,0.551688,0.481013,0.487342,0.560127,0.572785,0.086498,0.069620,0.464135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.373550
397,watch9_rotated_-270,0.504861,0.503472,0.329167,0.318056,0.482639,0.538194,0.048611,0.062500,0.465278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.303472
398,watch9_rotated_-90,0.495139,0.496528,0.329167,0.318056,0.517361,0.461806,0.048611,0.062500,0.534722,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.325000
